In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [40]:
a=pd.read_csv("C:/Users/srbeh/Desktop/export5.csv",low_memory=False)

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras

In [47]:
df1=a

In [49]:
df=df1/100000
df

,X [ m ],Y [ m ],Z [ m ],Approximated Mass Flow [ kg s^-1 ],Boundary Heat Flux Sensible [ W m^-2 ],Connectivity Number,Density [ kg m^-3 ],Domain Node Number [ ],Edge Length Ratio,Element Volume Ratio,...,Wall Adjacent Temperature [ K ],Wall Heat Flux [ W m^-2 ],Wall Radiative Heat Flux [ W m^-2 ],Wall Shear [ Pa ],Wall Shear X [ Pa ],Wall Shear Y [ Pa ],Wall Shear Z [ Pa ],X [ m ],Y [ m ].1,Z [ m ].1
0,5.740000e-09,6.930000e-09,0.000003,0.0,0.0,0.00002,0.00998,0.00001,0.000028,0.000010,...,0.00341,0.000,0.0,0.000401,-3.700000e-07,-4.340000e-07,-0.000401,5.740000e-09,6.930000e-09,0.000003
1,4.820000e-09,7.600000e-09,0.000003,0.0,0.0,0.00002,0.00998,0.00002,0.000028,0.000010,...,0.00341,0.000,0.0,0.000397,-3.020000e-07,-4.710000e-07,-0.000397,4.820000e-09,7.600000e-09,0.000003
2,4.820000e-09,7.600000e-09,0.000003,0.0,0.0,0.00004,0.00998,0.00003,0.000028,0.000010,...,0.00341,-0.986,0.0,0.001790,-1.330000e-06,-2.090000e-06,-0.001790,4.820000e-09,7.600000e-09,0.000003
3,5.740000e-09,6.930000e-09,0.000003,0.0,0.0,0.00004,0.00998,0.00004,0.000028,0.000010,...,0.00341,-0.994,0.0,0.001810,-1.630000e-06,-1.920000e-06,-0.001810,5.740000e-09,6.930000e-09,0.000003
4,6.560000e-09,6.160000e-09,0.000003,0.0,0.0,0.00002,0.00998,0.00005,0.000028,0.000010,...,0.00341,0.000,0.0,0.000405,-4.350000e-07,-3.930000e-07,-0.000405,6.560000e-09,6.160000e-09,0.000003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26662,-4.560000e-09,1.610000e-09,0.000002,0.0,0.0,0.00008,0.00998,0.26663,0.000052,0.000011,...,0.00000,0.000,0.0,0.000000,0.000000e+00,0.000000e+00,0.000000,-4.560000e-09,1.610000e-09,0.000002
26663,-4.800000e-09,1.050000e-09,0.000002,0.0,0.0,0.00008,0.00998,0.26664,0.000052,0.000012,...,0.00000,0.000,0.0,0.000000,0.000000e+00,0.000000e+00,0.000000,-4.800000e-09,1.050000e-09,0.000002
26664,-5.030000e-09,4.790000e-10,0.000002,0.0,0.0,0.00008,0.00998,0.26665,0.000052,0.000014,...,0.00000,0.000,0.0,0.000000,0.000000e+00,0.000000e+00,0.000000,-5.030000e-09,4.790000e-10,0.000002
26665,-5.180000e-09,-1.220000e-10,0.000002,0.0,0.0,0.00008,0.00998,0.26666,0.000056,0.000014,...,0.00000,0.000,0.0,0.000000,0.000000e+00,0.000000e+00,0.000000,-5.180000e-09,-1.220000e-10,0.000002


In [50]:
xtrain=df.iloc[:,0:55]
ytrain=df.iloc[:,55:]

In [51]:
inputs=keras.Input(shape=(55,))
x1=keras.layers.Dense(6,activation=keras.activations.selu)(inputs)
x2=keras.layers.Dense(12,activation=keras.activations.selu)(x1)
x3=keras.layers.Dense(24,activation=keras.activations.selu)(x2)
x4=keras.layers.Dense(48,activation=keras.activations.selu)(x3)
x5=keras.layers.Dense(96,activation=keras.activations.selu)(x4)
x6=keras.layers.Dense(192,activation=keras.activations.selu)(x5)
x7=keras.layers.Dense(200,activation=keras.activations.selu)(x6)
x8=keras.layers.Dense(200,activation=keras.activations.selu)(x7)
x9=keras.layers.Dense(224,activation=keras.activations.selu)(x8)
outputs=keras.layers.Dense(60)(x9)
model=keras.Model(inputs,outputs)
optimizer=keras.optimizers.Adam(learning_rate=1e-3)
model.summary()
batchsize=1
a=tf.convert_to_tensor(xtrain,dtype='float32').numpy()
b=tf.convert_to_tensor(ytrain,dtype='float32').numpy()
traindataset=tf.data.Dataset.from_tensor_slices((a,b))
#traindataset=traindataset.shuffle(buffer_size=100).batch(batchsize)
traindataset=traindataset.batch(batchsize)
def my_loss_fn(y_true,y_pred,x_true):
    squared_difference=tf.sqrt((y_true-y_pred)**2)
    return tf.reduce_mean(squared_difference)
epochs=1
new_p=[]
new_v=[]
for epoch in range(epochs):
    print("\nStart of epoch %d"%(epoch,))
    for step,(xx,yy) in enumerate(traindataset):
        with tf.GradientTape(persistent=True) as tape:
            logits=model(xx,training=True)
            lossvalue=my_loss_fn(yy,logits,xx)
            #new_p.append(logits.numpy()[0][0])
            #new_v.append(logits.numpy()[0][1])
        print(step+1)
        print('loss',lossvalue)
        grads=tape.gradient(lossvalue,model.trainable_variables)
        optimizer.apply_gradients(zip(grads,model.trainable_variables))
        
print('Thank you for your patience')  

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 55)]              0         
                                                                 
 dense_70 (Dense)            (None, 6)                 336       
                                                                 
 dense_71 (Dense)            (None, 12)                84        
                                                                 
 dense_72 (Dense)            (None, 24)                312       
                                                                 
 dense_73 (Dense)            (None, 48)                1200      
                                                                 
 dense_74 (Dense)            (None, 96)                4704      
                                                                 
 dense_75 (Dense)            (None, 192)               1862

118
loss tf.Tensor(196.6077, shape=(), dtype=float32)
119
loss tf.Tensor(433.31152, shape=(), dtype=float32)
120
loss tf.Tensor(353.5341, shape=(), dtype=float32)
121
loss tf.Tensor(439.9647, shape=(), dtype=float32)
122
loss tf.Tensor(400.67325, shape=(), dtype=float32)
123
loss tf.Tensor(259.4384, shape=(), dtype=float32)
124
loss tf.Tensor(144.2686, shape=(), dtype=float32)
125
loss tf.Tensor(180.39536, shape=(), dtype=float32)
126
loss tf.Tensor(176.39224, shape=(), dtype=float32)
127
loss tf.Tensor(24731.217, shape=(), dtype=float32)
128
loss tf.Tensor(58407.2, shape=(), dtype=float32)
129
loss tf.Tensor(3716666400.0, shape=(), dtype=float32)
130
loss tf.Tensor(399999780.0, shape=(), dtype=float32)
131
loss tf.Tensor(19757.668, shape=(), dtype=float32)
132
loss tf.Tensor(53250.152, shape=(), dtype=float32)
133
loss tf.Tensor(3216666400.0, shape=(), dtype=float32)
134
loss tf.Tensor(346666370.0, shape=(), dtype=float32)
135
loss tf.Tensor(122.14035, shape=(), dtype=float32)
136
los

273
loss tf.Tensor(97.09082, shape=(), dtype=float32)
274
loss tf.Tensor(179.96812, shape=(), dtype=float32)
275
loss tf.Tensor(212.61238, shape=(), dtype=float32)
276
loss tf.Tensor(26975.441, shape=(), dtype=float32)
277
loss tf.Tensor(57140.67, shape=(), dtype=float32)
278
loss tf.Tensor(3866666500.0, shape=(), dtype=float32)
279
loss tf.Tensor(413333100.0, shape=(), dtype=float32)
280
loss tf.Tensor(113.8602, shape=(), dtype=float32)
281
loss tf.Tensor(52.143234, shape=(), dtype=float32)
282
loss tf.Tensor(64.79889, shape=(), dtype=float32)
283
loss tf.Tensor(149.41428, shape=(), dtype=float32)
284
loss tf.Tensor(210.42787, shape=(), dtype=float32)
285
loss tf.Tensor(217.12225, shape=(), dtype=float32)
286
loss tf.Tensor(134.56615, shape=(), dtype=float32)
287
loss tf.Tensor(170.47739, shape=(), dtype=float32)
288
loss tf.Tensor(70.52102, shape=(), dtype=float32)
289
loss tf.Tensor(209.16139, shape=(), dtype=float32)
290
loss tf.Tensor(176.9365, shape=(), dtype=float32)
291
loss tf

428
loss tf.Tensor(46139.016, shape=(), dtype=float32)
429
loss tf.Tensor(80802.07, shape=(), dtype=float32)
430
loss tf.Tensor(4716667000.0, shape=(), dtype=float32)
431
loss tf.Tensor(509999740.0, shape=(), dtype=float32)
432
loss tf.Tensor(118.88309, shape=(), dtype=float32)
433
loss tf.Tensor(83.75802, shape=(), dtype=float32)
434
loss tf.Tensor(61.633167, shape=(), dtype=float32)
435
loss tf.Tensor(129.55708, shape=(), dtype=float32)
436
loss tf.Tensor(174.66382, shape=(), dtype=float32)
437
loss tf.Tensor(204.53418, shape=(), dtype=float32)
438
loss tf.Tensor(151.29996, shape=(), dtype=float32)
439
loss tf.Tensor(117.543846, shape=(), dtype=float32)
440
loss tf.Tensor(60.34796, shape=(), dtype=float32)
441
loss tf.Tensor(242.79887, shape=(), dtype=float32)
442
loss tf.Tensor(243.29283, shape=(), dtype=float32)
443
loss tf.Tensor(184.18486, shape=(), dtype=float32)
444
loss tf.Tensor(152.77303, shape=(), dtype=float32)
445
loss tf.Tensor(153.58812, shape=(), dtype=float32)
446
los

583
loss tf.Tensor(121.27811, shape=(), dtype=float32)
584
loss tf.Tensor(89.04296, shape=(), dtype=float32)
585
loss tf.Tensor(141.3786, shape=(), dtype=float32)
586
loss tf.Tensor(247.97472, shape=(), dtype=float32)
587
loss tf.Tensor(354.60437, shape=(), dtype=float32)
588
loss tf.Tensor(646.1837, shape=(), dtype=float32)
589
loss tf.Tensor(981.4777, shape=(), dtype=float32)
590
loss tf.Tensor(1676.8837, shape=(), dtype=float32)
591
loss tf.Tensor(2392.6733, shape=(), dtype=float32)
592
loss tf.Tensor(3222.0234, shape=(), dtype=float32)
593
loss tf.Tensor(7299.532, shape=(), dtype=float32)
594
loss tf.Tensor(9395.113, shape=(), dtype=float32)
595
loss tf.Tensor(181487.72, shape=(), dtype=float32)
596
loss tf.Tensor(106142.086, shape=(), dtype=float32)
597
loss tf.Tensor(7000000000.0, shape=(), dtype=float32)
598
loss tf.Tensor(758333100.0, shape=(), dtype=float32)
599
loss tf.Tensor(134.09697, shape=(), dtype=float32)
600
loss tf.Tensor(165.1082, shape=(), dtype=float32)
601
loss tf

...

26045
loss tf.Tensor(296.46884, shape=(), dtype=float32)
26046
loss tf.Tensor(2099.8008, shape=(), dtype=float32)
26047
loss tf.Tensor(4633.1333, shape=(), dtype=float32)
26048
loss tf.Tensor(3716.4607, shape=(), dtype=float32)
26049
loss tf.Tensor(711.44867, shape=(), dtype=float32)
26050
loss tf.Tensor(49.942852, shape=(), dtype=float32)
26051
loss tf.Tensor(25.602467, shape=(), dtype=float32)
26052
loss tf.Tensor(83.92607, shape=(), dtype=float32)
26053
loss tf.Tensor(171.41347, shape=(), dtype=float32)
26054
loss tf.Tensor(191.4111, shape=(), dtype=float32)
26055
loss tf.Tensor(0.44710883, shape=(), dtype=float32)
26056
loss tf.Tensor(0.43960482, shape=(), dtype=float32)
26057
loss tf.Tensor(0.48754868, shape=(), dtype=float32)
26058
loss tf.Tensor(0.432187, shape=(), dtype=float32)
26059
loss tf.Tensor(0.49869022, shape=(), dtype=float32)
26060
loss tf.Tensor(0.48613256, shape=(), dtype=float32)
26061
loss tf.Tensor(0.47787878, shape=(), dtype=float32)
26062
loss tf.Tensor(0.49073

26188
loss tf.Tensor(0.29454708, shape=(), dtype=float32)
26189
loss tf.Tensor(0.28928947, shape=(), dtype=float32)
26190
loss tf.Tensor(0.28074324, shape=(), dtype=float32)
26191
loss tf.Tensor(0.275475, shape=(), dtype=float32)
26192
loss tf.Tensor(0.32014814, shape=(), dtype=float32)
26193
loss tf.Tensor(0.39989716, shape=(), dtype=float32)
26194
loss tf.Tensor(0.42846596, shape=(), dtype=float32)
26195
loss tf.Tensor(0.36691126, shape=(), dtype=float32)
26196
loss tf.Tensor(0.32288286, shape=(), dtype=float32)
26197
loss tf.Tensor(0.32116643, shape=(), dtype=float32)
26198
loss tf.Tensor(0.33691195, shape=(), dtype=float32)
26199
loss tf.Tensor(0.3219775, shape=(), dtype=float32)
26200
loss tf.Tensor(0.37549135, shape=(), dtype=float32)
26201
loss tf.Tensor(0.3862146, shape=(), dtype=float32)
26202
loss tf.Tensor(0.43691102, shape=(), dtype=float32)
26203
loss tf.Tensor(0.4584566, shape=(), dtype=float32)
26204
loss tf.Tensor(0.26638627, shape=(), dtype=float32)
26205
loss tf.Tenso

26334
loss tf.Tensor(0.11076285, shape=(), dtype=float32)
26335
loss tf.Tensor(0.31333587, shape=(), dtype=float32)
26336
loss tf.Tensor(0.0788587, shape=(), dtype=float32)
26337
loss tf.Tensor(0.64457995, shape=(), dtype=float32)
26338
loss tf.Tensor(2.861787, shape=(), dtype=float32)
26339
loss tf.Tensor(0.09297669, shape=(), dtype=float32)
26340
loss tf.Tensor(0.1771396, shape=(), dtype=float32)
26341
loss tf.Tensor(0.097632855, shape=(), dtype=float32)
26342
loss tf.Tensor(0.51682526, shape=(), dtype=float32)
26343
loss tf.Tensor(0.14638111, shape=(), dtype=float32)
26344
loss tf.Tensor(1.4717927, shape=(), dtype=float32)
26345
loss tf.Tensor(1.1208488, shape=(), dtype=float32)
26346
loss tf.Tensor(7.637504, shape=(), dtype=float32)
26347
loss tf.Tensor(77.60103, shape=(), dtype=float32)
26348
loss tf.Tensor(181.59259, shape=(), dtype=float32)
26349
loss tf.Tensor(143.91402, shape=(), dtype=float32)
26350
loss tf.Tensor(49.91842, shape=(), dtype=float32)
26351
loss tf.Tensor(5.2418

26478
loss tf.Tensor(0.11224764, shape=(), dtype=float32)
26479
loss tf.Tensor(0.08796488, shape=(), dtype=float32)
26480
loss tf.Tensor(0.12203191, shape=(), dtype=float32)
26481
loss tf.Tensor(11.432258, shape=(), dtype=float32)
26482
loss tf.Tensor(0.0876722, shape=(), dtype=float32)
26483
loss tf.Tensor(0.063709766, shape=(), dtype=float32)
26484
loss tf.Tensor(0.069532, shape=(), dtype=float32)
26485
loss tf.Tensor(0.058765355, shape=(), dtype=float32)
26486
loss tf.Tensor(0.09166256, shape=(), dtype=float32)
26487
loss tf.Tensor(0.07045336, shape=(), dtype=float32)
26488
loss tf.Tensor(0.07965334, shape=(), dtype=float32)
26489
loss tf.Tensor(1.2439824, shape=(), dtype=float32)
26490
loss tf.Tensor(2.2915502, shape=(), dtype=float32)
26491
loss tf.Tensor(10.697002, shape=(), dtype=float32)
26492
loss tf.Tensor(0.1919145, shape=(), dtype=float32)
26493
loss tf.Tensor(0.13697536, shape=(), dtype=float32)
26494
loss tf.Tensor(0.44308573, shape=(), dtype=float32)
26495
loss tf.Tensor

26622
loss tf.Tensor(126.00882, shape=(), dtype=float32)
26623
loss tf.Tensor(114.500206, shape=(), dtype=float32)
26624
loss tf.Tensor(126.32353, shape=(), dtype=float32)
26625
loss tf.Tensor(836.482, shape=(), dtype=float32)
26626
loss tf.Tensor(3716.4802, shape=(), dtype=float32)
26627
loss tf.Tensor(178.14334, shape=(), dtype=float32)
26628
loss tf.Tensor(948.1424, shape=(), dtype=float32)
26629
loss tf.Tensor(2866.465, shape=(), dtype=float32)
26630
loss tf.Tensor(3716.4585, shape=(), dtype=float32)
26631
loss tf.Tensor(1766.4558, shape=(), dtype=float32)
26632
loss tf.Tensor(271.453, shape=(), dtype=float32)
26633
loss tf.Tensor(21.609224, shape=(), dtype=float32)
26634
loss tf.Tensor(77.861435, shape=(), dtype=float32)
26635
loss tf.Tensor(28.08907, shape=(), dtype=float32)
26636
loss tf.Tensor(273.08002, shape=(), dtype=float32)
26637
loss tf.Tensor(656.4124, shape=(), dtype=float32)
26638
loss tf.Tensor(303.0801, shape=(), dtype=float32)
26639
loss tf.Tensor(120.74472, shape=(

In [1]:
model.summary()

NameError: name 'model' is not defined

In [ ]:
 def loSS_fn(self, x, y, t):
        lambda_1 = self.lambda_1
        lambda_2 = self.lambda_2
        
        psi_and_p = self.neural_net(tf.concat([x,y,t], 1), self.weights, self.biases)  # basically eithi GradientTape pakeibaku padiba
        psi = psi_and_p[:,0:1]
        p = psi_and_p[:,1:2]
        
        u = tf.gradients(psi, y)[0]
        v = -tf.gradients(psi, x)[0]    
        
        u_t = tf.gradients(u, t)[0]
        u_x = tf.gradients(u, x)[0]
        u_y = tf.gradients(u, y)[0]
        u_xx = tf.gradients(u_x, x)[0]
        u_yy = tf.gradients(u_y, y)[0]
        
        v_t = tf.gradients(v, t)[0]
        v_x = tf.gradients(v, x)[0]
        v_y = tf.gradients(v, y)[0]
        v_xx = tf.gradients(v_x, x)[0]
        v_yy = tf.gradients(v_y, y)[0]
        
        p_x = tf.gradients(p, x)[0]
        p_y = tf.gradients(p, y)[0]

        f_u = u_t + lambda_1*(u*u_x + v*u_y) + p_x - lambda_2*(u_xx + u_yy) 
        f_v = v_t + lambda_1*(u*v_x + v*v_y) + p_y - lambda_2*(v_xx + v_yy)
        
        return u, v, p, f_u, f_v